*Marcello Victorino* <br>
*04/30/2019* -  

# Introduction
This project is part of a requirement to graduate in the Udacity's Data Analyst Nanodegree (*DAND*).

It provides the opportunity to implement Data Wrangling in practice by gathering data from different sources, assessing it for quality and tidiness issues and then promote the necessary cleaning task - programatically.

Finally, once the data is properly cleaned and stored, a brief analysis is conducted with visualizations, highlighting interesting insights.

The data for this project was provided in partnership with the **WeRateDogs** channel from twitter, containing over 5,000 observations about dogs.

In [1]:
import pandas as pd
import numpy as np

## Gather

In [2]:
# Gathering Twitter Enhanced Archive data
df_archive = pd.read_csv('twitter-archive-enhanced-2.csv')
df_archive.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


In [3]:
# Gathering Image Predictions for Dog Breed - Available online
import requests
import os

# Avoid redownloading if file already saved locally
if 'dog_breed.txt' in os.listdir():
    pass

else:
    url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

    # Accessing file online
    r = requests.get(url)

    # Saving content locally
    with open('dog_breed.txt', 'wb') as fh:
        fh.write(r.content)

# Reading file as Dataframe
if 'dog_breed.csv' in os.listdir():
    df_breed = pd.read_csv('dog_breed.csv')

else:
    df_breed = pd.read_csv('dog_breed.txt', sep='\t')
    df_breed.to_csv('dog_breed.csv', index=False)

df_breed.tweet_id = df_breed.tweet_id.astype(str)
df_breed.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


In [4]:
# Gathering data from Twitter API
import tweepy
from twitter_secret_credentials import Twitter_API_Authenticate # personal script

# Authenticating connection to Twitter API
api = Twitter_API_Authenticate() # wait_on_rate_limit=True, wait_on_rate_limit_notify=True

In [5]:
# Retrieve data from Twitter ID (JSON format)
import json
from tqdm import tqdm_notebook as progressbar

# Avoid redownloading if file already saved locally
if 'tweet_json.txt' in os.listdir():
    print('JSON data has already been downloaded.')
    pass

else:
    # Retrieve data from Twitter API and save locally
    fails = []
    count = 0
    with open('tweet_json.txt', 'w') as file:
        for tweet_id in progressbar(df_archive.tweet_id[:]):
            count += 1
            try:
                tweet = api.get_status(tweet_id, tweet_mode='extended')
                json.dump(tweet._json, file)
                file.write('\n') # important to separate each tweet  

            except:
                fails.append(tweet_id)

    fail_percentage = len(fails)/count
    print(f'Successfully read: {(1 - fail_percentage):.0%}') # 19 tweets could not be read | 25 minutes

JSON data has already been downloaded.


In [6]:
# Actually working with the JSON data extracted

# Avoid duplicating work if data already parsed and saved locally
if 'tweet_parsed_data.csv' in os.listdir():
    print('Data already parsed and saved.')
    df_tweet = pd.read_csv('tweet_parsed_data.csv')

else:
    with open('tweet_json.txt', 'r') as file:
        tweet_jsons = file.readlines()

    # Iterating over each individual tweet
    tweet_data = list()

    for tweet in tweet_jsons:
        data = dict()

        js = json.loads(tweet) # Reading each tweet string as JSON
        
        # Skip text starting with "RT" or "@"
        if js['full_text'].startswith(('RT', '@')):
            continue

    #         print(json.dumps(js, indent=4)) # Pretty printing JSON
        data['tweet_id'] = js['id_str']
        data['created'] = js['created_at']
        data['retweet'] = js['retweet_count']
        data['favorite'] = js['favorite_count']
        data['text'] = js['full_text']

        tweet_data.append(data)

    # Reading data into Dataframe
    df_tweet = pd.DataFrame(tweet_data, columns=data.keys())

    # Extract dog's name
    df_tweet['name'] = df_tweet.text.str.extract(' ([A-Z][a-z]*)')

    # Extract rating
    df_tweet['rate'] = df_tweet.text.str.extract('([0-9]*)/[0-9]{2}')

    # Transforming df_tweet.created as Datetime
    df_tweet.created = pd.to_datetime(df_tweet.created)

    # Extract datetime from Created feature
    df_tweet['year'] = df_tweet.created.dt.year
    df_tweet['month'] = df_tweet.created.dt.month
    df_tweet['weekday'] = df_tweet.created.dt.day_name()
    df_tweet['hour'] = df_tweet.created.dt.hour

    # Saving it locally
    df_tweet.to_csv('tweet_parsed_data.csv', index=False)

df_tweet.tweet_id = df_tweet.tweet_id.astype(str)
df_tweet.head(2)

Data already parsed and saved.


,tweet_id,created,retweet,favorite,text,name,rate,year,month,weekday,hour
0,892420643555336193,2017-08-01 16:23:56+00:00,8197,37569,This is Phineas. He's a mystical boy. Only eve...,Phineas,13.0,2017,8,Tuesday,16
1,892177421306343426,2017-08-01 00:17:27+00:00,6060,32304,This is Tilly. She's just checking pup on you....,Tilly,13.0,2017,8,Tuesday,0


# TODO:
+ check tidiness issue
    - df_archive: doggo, fluffer etc should be melted into 1 column 'category'
    - Separate table for Dog information: tweet_id, dog name, dog breed (breed + confidence), rating
    - Separate Table for Tweet: tweet_id, creted, retweet count, favorite count, year+month etc.

## Data Assessment and Cleaning

In [7]:
df_archive.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

### 1) Twitter Archived Enhanced Dataset
#### Define
Tidiness:
+ Multiple stage categories (puppo, fluffer etc.) in separate columns -> should be melted into a single Category variable

Quality:
+ tweet_id stored as integer -> should be as string
+ Many columns filled with missing values or useless data -> remove 7 columns: ['in_reply_to_status_id', 'in_reply_to_user_id', 'source', 'retweeted_status_id', retweeted_status_user_id', retweeted_status_timestamp', 'expanded_urls']
+ Replace 'None' entries with `np.NaN`

#### Code

In [ ]:
# Tidy: grouping many columns into a single one
# Obs: pd.melt did not accomplish what I wanted

df_archive['category'] = np.where(df_archive.doggo != 'None', 'doggo',
                            np.where(df_archive.floofer != 'None', 'floofer',
                            np.where(df_archive.pupper != 'None', 'puper',
                            np.where(df_archive.puppo != 'None', 'puppo', np.NaN))))

# Quality: Dropping irrelavant or columns already available in other dataset
df_archive.drop(columns=['in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'doggo', 'floofer', 'pupper', 'puppo'], inplace=True)

# Quality: Store ID as string for later joining reference
df_archive.tweet_id = df_archive.tweet_id.astype(str)

# Quality: Replace 'None' entries with NaN
df_archive.replace('None', np.nan);

#### Test

In [9]:
df_archive.loc[8:13]

,tweet_id,name,category
8,890609185150312448,Zoey,nan
9,890240255349198849,Cassie,doggo
10,890006608113172480,Koda,nan
11,889880896479866881,Bruno,nan
12,889665388333682689,None,puppo
13,889638837579907072,Ted,nan


### 2) Dog Breed
#### Define
Tidiness: N/A

Quality:
+ Remove irrelevant columns -> ['p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog']
+ Remove entries with Confidence Score <= 59% -> horribly inaccurate dog breed
+ Remove entries Not Dog -> Dog = False
+ Store tweet_id as string

In [10]:
df_breed.drop(columns=['p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], inplace=True)

In [14]:
# Remove observations with confidence score lower than 59% -> incorrect dog breed
df_breed = df_breed.query('p1_conf >= 0.59')

# Remove observations not containg a dog
df_breed = df_breed.query('p1_dog == True')
df_breed.head()

# Store tweet_id as string
df_breed.tweet_id = df_breed.tweet_id.astype(str)

len(df_breed) # Total of 810 entries with "Good Accurate" data

810

#### Test

In [20]:
# Test for Confidence Score
assert (df_breed.p1_conf >= 0.59).all()

In [21]:
# Test for is Dog
assert (df_breed.p1_dog == True).all()

In [33]:
# Test if tweet_id is string (object in Pandas)
assert df_breed.tweet_id.dtype.kind == 'O'

In [15]:
df_breed.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True


In [22]:
df_tweet.head(2)

,tweet_id,created,retweet,favorite,text,name,rate,year,month,weekday,hour
0,892420643555336193,2017-08-01 16:23:56+00:00,8197,37569,This is Phineas. He's a mystical boy. Only eve...,Phineas,13.0,2017,8,Tuesday,16
1,892177421306343426,2017-08-01 00:17:27+00:00,6060,32304,This is Tilly. She's just checking pup on you....,Tilly,13.0,2017,8,Tuesday,0


### 3) Scraped Twitter API
#### Define
Tidiness: N/A

Quality:


# Data Wrangling Report
This part of the project consisted in **Gathering** data from 3 different sources and pre-storing them locally, and then **Assessing** and **Cleaning** as necessary (*Quality* and *Tidiness* issues).

Once cleaned, the data was saved locally as its "final" version, making sure each table contains information regarding a specific subject.

## Gathering
+ **Twitter Archived Enhanced**: this `.csv` file was made available and easily incorporated using the `pd.read_csv` function

+ **Dog Breed**: the data was available online in a specific `url`, requiring to access it via the `requests` library and writing it to a local file. Since this data is *tabular separated*, it was incorporated via the `pd.read_csv` function, specifying the parameter `sep='\t'`, to read from tabular structure. The pre-version of this file was saved as **dog_breed.csv**

+ **Twitter Specific Data**: this data was accessed via the *Twitter API*, using the wrapper library `tweepy` to easy authentication and information retrieval. With the unique *Tweet ID* for each observation, it was possible to extract additional data from each tweet by parsing the *JSON* format returned by the API (using the `json` package). More specifically, we obtained the following:
    - *Date of Creation*: allowing to extract further data such as `year`, `month`, `day of week`, and `hour of day`;
    - *Dog Name*: using `Regex` to find and extract the majority of dog names (note: not all tweets contained this information)
    - *Retweet Count*: amount of times a specific tweet was shared by others
    - *Favorite Count*: number of times a specific tweet was "liked" by users

## Assessing & Cleaning
### Tidiness Issues
1. asd
2. asd

### Quality Issues
1. asd
2. asd
3. asd
4. asd
5. asd
6. asd
7. asd
8. asd
9. ad
10. asd